In [1]:
import numpy as np
import scipy
import scipy.special
import itertools
import pandas as pd
import math
import copy
from math import comb
from sympy import Matrix, symbols, Poly,degree
from functools import partial
import dask
import dask.dataframe as dd
from sympy.utilities.lambdify import lambdify
import time

# Greedy Aproximation via Energy Increment

This is an implementation of the Energy Increment algorithm in the paper https://arxiv.org/abs/2207.12529.  The algorithm uses following sub-routines

* Computation of $L_p$ norms of symmetric tensors using the quadrature rule proved by Cristancho and Velasco
* Sampling a collection of vectors on $n$-sphere $S^{n-1}$ and evaluating a symmetric tensor on these vectors 
* Creating a flag of subspaces and projecting a symmetric tensor on these subspaces. Eventually, these projections create the low-rank approximation.


## S. Cristancho and M. Velasco quadrature rule implementation into Python

We start implementing in S. Cristancho and M. Velasco, Harmonic hierarchies for polynomial optimization, https://arxiv.org/abs/2202.12865, for computing $L_p$-norms of symmetric tensors. We want to note from the beginning that we will use monomial index notation in this code. We recall that by a quadrature formula of algebraic degree $2t$ for the Borel measure $\mu$ on $\mathbb{S}^{n-1}$ Cristancho and Velasco paper means a pair $(X,W)$ where $X\subset \mathbb{S}^{n-1}$ and $W: X \to \mathbb{R}_{>0}$ is a function satisfying the equality
\begin{equation}
\label{spherical_formula}
    \displaystyle\int_{\mathbb{S}^{n-1}} f(y) \, d\mu(y)
    =
    \sum\limits_{x\in X} W(x) f(x)
\end{equation}
for every homogeneous polynomial of even degree $2t$.


### Computing the pair $(X,W)$ in quadrature rule

The rule uses volume of $n$-dimensional ball and surface area of $n$-dimensional sphere, they are computed below.

#### Volume of the $n$-dimensional ball

In [1]:
def ball_volume(n:int) -> float:
    
    """
    Parameter:
    
    n: int. Dimension of the ball.
    
    Output:
    
    V: float. Volume of the n-dimensional ball.
    """
    
    V = 1
    if n==1:
        V = 2
    elif n>1:
        V = (2*np.pi/n)*ball_volume(n-2)
    elif n<0:
        raise ValueError("The dimension n cannot be lower than 0")
    return V

#### Surface Area of the $n$-dimensional sphere

In [3]:
def sphere_surface_area(n:int) -> float:
    
    """
    Parameter:
    
    n: int. Dimension of the ball.
    
    Output:
    
    return: float. Surface area of the n-dimensional sphere.
    """
    
    return (n+1)*ball_volume(n+1)

#### Spherical quadrature function

We now we provice S. Cristancho and M. Velasco Julia package's implementation in python.

In [4]:
def custom_flatten(x):
    
    """
    Parameter:
    
    x: array. List of two elements.
    
    Output:
    
    result: array. The second element of the list has a transformation.
    """
    
    t1 = x[0]
    t2 = np.sqrt(1-x[0]**2)*x[1]
    result = np.append(np.array(t1), np.array(t2))

    return result

def spherical_quadrature(n:int, deg:int):
    
    """
    Parameters:
    
    n: int. Dimension of the space.    
    deg: int. Degree. Must be an even number.
    
    Output:
    
    (X, WX): (array, array). Pair (X,W) for the cubature formula.
    """
    
    X = np.array([1.,-1.])
    WX = np.array([1.,1.])
    if n > 1:
        Y, WY = spherical_quadrature(n-1,deg)
        Z, WZ = scipy.special.roots_jacobi(deg,(n-3)/2,(n-3)/2)
        X = np.array([custom_flatten(comb) for comb in itertools.product(Z,Y)])
        WX = np.array([np.prod(comb) for comb in itertools.product(WZ,WY)])
    elif n < 1:
        raise ValueError("dimension argument must be positive")
        
    return X,WX

##### Example

In [5]:
spherical_quadrature(3,4)

(array([[-0.86113631, -0.46967645,  0.19454636],
        [-0.86113631, -0.46967645, -0.19454636],
        [-0.86113631, -0.19454636,  0.46967645],
        [-0.86113631, -0.19454636, -0.46967645],
        [-0.86113631,  0.19454636,  0.46967645],
        [-0.86113631,  0.19454636, -0.46967645],
        [-0.86113631,  0.46967645,  0.19454636],
        [-0.86113631,  0.46967645, -0.19454636],
        [-0.33998104, -0.86884614,  0.35988786],
        [-0.33998104, -0.86884614, -0.35988786],
        [-0.33998104, -0.35988786,  0.86884614],
        [-0.33998104, -0.35988786, -0.86884614],
        [-0.33998104,  0.35988786,  0.86884614],
        [-0.33998104,  0.35988786, -0.86884614],
        [-0.33998104,  0.86884614,  0.35988786],
        [-0.33998104,  0.86884614, -0.35988786],
        [ 0.33998104, -0.86884614,  0.35988786],
        [ 0.33998104, -0.86884614, -0.35988786],
        [ 0.33998104, -0.35988786,  0.86884614],
        [ 0.33998104, -0.35988786, -0.86884614],
        [ 0.33998104

## Rank one symmetric tensors $v \otimes \cdots \otimes v$

We recall that symmetric tensors can be identified with homogenous polynomials. Therefore, for convenience, we work with polynomials of the type $p(x)= c \cdot \langle v, x \rangle^d$ to represent rank one symmetric tensors of the type $v \otimes \cdots \otimes v$.

In [6]:
class custom_polynomial:
    
    def __init__(self,const:int, v:list, d:int):
        
        """
        Parameters:
        
        self: Parameter in instance method.
        const: int. Constant multiplying the inner product on the expression above.
        v: list. Vector of real numbers.
        d: int. Degree of the polynomial. This must be an even number.
        """
        
        self.num_var = len(v)
        self.variables = symbols(" ".join([f"x{i}" for i in range(1,self.num_var + 1)]))
        self.expr = const*(Matrix(list(self.variables)).dot((Matrix(list(v))))**d)
        self.degree = d
        self.f = lambdify(list(self.variables),self.expr)
        
### The evaluate function evaluates a given polynomial at a given point.
    
    def evaluate(self,w):
        
        """
        Parameters:
        
        self: Parameter in instance method. Polynomial defined in the class.
        w: list. Point at which the polynomial is evaluated.
        
        Output:
        
        result: float. Evaluation of the polynomial at w.
        """
        
        if self.num_var != len(w):
            raise Exception("This polynomial cannot be evaluated")
            
        result = self.f(*list(w))
        
        return float(result)
    
### Using the spherical quadrature function created above, we compute the L_r-norm of the polynomial.
    
    def Lr_norm(self,X,WX,r):
        
        """
        Parameters:
        
        self: Parameter in instance method. Polynomial defined in the class.
        X: list. List of elements in X corresponding to the cubature formula.
        WX: list. List of elements in WX corresponding to the cubature formula.
        r: int. Point at which the polynomial is evaluated.
        
        Output:
        
        return: float. L_r-norm of the polynomial.
        """
        
        c = sphere_surface_area(self.num_var-1)
        
        evaluations = parallelized_evaluation(self,X)
        evaluations = np.array(evaluations)**r
        
        return ((1/c) * np.sum(WX*(evaluations)))**(1/r)

For convenience we include a function calling the evaluate above.

In [7]:
def evaluate_pol(poly,v):
    
    """
    Parameters:
    
    poly: custom_polynomial. Polynomial to be evaluated.
    v: list. Point we the polynomial is evaluated.
    
    Output:
    
    fv: float. Evaluation of the polynomial at v.
    """

    fv = poly.evaluate(v)
    return fv

In addition, to accelarate the process, we include the parallelized_evaluation. This allows us to evaluate a polynomial at different points in a parallelized manner.

In [8]:
def parallelized_evaluation(poly,V,npartitions = 8):

    """
    Parameters:
    
    poly: custom_polynomial. Polynomial in the class custom_polynomial.    
    V: list. Points at which the polynomial is evaluated.
    npartitions: . Number of partitions to be used.
    
    Output:
    
    fV: list. Evaluations of the polynomial at each element of V.
    """
    
    df = pd.DataFrame(V)
    df = dd.from_pandas(df,npartitions = npartitions)
    
    fV = df.apply(lambda row: evaluate_pol(poly,row), axis = 1, meta = (None, 'float64'))
    
    fV = fV.compute()
    
    return fV.values

##### Example

In [9]:
deg = 4
r = 4
vector = [1,2,3]
const = 7

poly = custom_polynomial(const,vector,deg)

poly.expr

7*(x1 + 2*x2 + 3*x3)**4

In [10]:
X, WX = spherical_quadrature(3,4)
lrnorm = poly.Lr_norm(X, WX, r)

lrnorm

697.5393316313748

## Perturbed Symmetric Tensors

We create symmetric tensors as follows: we first form a low-rank input $\sum\limits_{i=1}^m c_i \langle v_i,x \rangle ^d$, then we perturb this input
with a high rank symmetric tensor to make it a high-rank tensor. In the end, we create high-rank symmetric tensors that have
$\epsilon$ distance to a low-rank input.

In [11]:
def polynomial_sum(polynomial_list, eps):
    """
    Parameter:
    
    polynomial_list: list. List of polynomials in the class custom_polynomial.
    
    eps: float. The epsilon perturbation to introduce.
    
    Output:
    
    output_poly: custom_polynomial. Polynomial resulting from the sum of all polynomials given in the list.
    """
    
    deg = polynomial_list[0].degree
    variables = polynomial_list[0].variables
    
    output_poly = copy.copy(polynomial_list[0])
    
    output_expr = 0
    for i in range(len(polynomial_list)):
        output_expr += polynomial_list[i].expr
    
    aux_poly = eps*0.5*sum(variables[i]**2 for i in range(len(variables)))**(deg*0.5)
    
    output_poly.expr = output_expr + aux_poly
    output_poly.f = lambdify(list(output_poly.variables),output_poly.expr)
    
    return output_poly

##### Example

In [12]:
m= 2
n = 3
deg = 2
vectors = np.random.randn(m, n)
vectors /= np.linalg.norm(vectors, axis=1)[:, None]
consts = np.random.randn(m)

polynomial_list = [custom_polynomial(consts[i],vectors[i],deg) for i in range(m)]

poly = polynomial_sum(polynomial_list, 0.1)

poly.expr

-0.477995365510691*(0.0758187359313211*x1 + x2 + 0.415177202076683*x3)**2 + 0.121471220538655*(0.751366977446341*x1 + 0.393801346558698*x2 - x3)**2 + 0.05*(x1**2 + x2**2 + x3**2)**1.0

## Finding $v \in \mathbb{S}^{n-1}$ such that $\frac{1}{2}||f||_r \leq |f(v)|$ for a symmetric tensor $f$

We define a function which samples vectors on the sphere.

In [13]:
def sample_spherical(n, size):

    """
    Parameters:
    
    n: int. Number of variables.
    size: int. Number of points to sample.
    
    Output:
    
    V: list. Vector of points sampled on the (n-1)-dimensional sphere.
    """
    
    V = np.random.randn(size, n)
    V /= np.linalg.norm(V, axis=1)[:, None]
    return V

 Note that for reasonably large $n$, the sample size does not depend on $n$. The sample size increases with respect to $r$ in $L_r$-norm and with respect to degree exponentially. We determined the needed size for a high probability success in our paper, https://arxiv.org/pdf/2207.12529.pdf, below it is recorded.

In [14]:
def sample_size(n, deg, r, accu):
    
    """
    Parameters:
    
    n: int. Number of variables.
    deg: int. Degree of the polynomial. Must be an even number.
    r: int. We need to specify here which L_r-norm we are using.
    accu: float. Number between 0 and 1 determining the level of accuracy.
    
    Output:
    
    return: int. Size needed to achieve the condition with the given level of accuracy.
    """
    
    C = 2
    t = -np.log(1-accu)
    a = (C*r)**(deg/2)
    b = (comb(r*deg + n - 1 , r*deg))**(1/(2*r))
    alpha = np.minimum(a,b)
    return math.ceil(t * (alpha)**(2*r))

We provide the function that verifies the desired condition holds for a vector. If after sampling the condition does not hold for any vector, the function samples again. This allows us to change the sample size as we wish and still have a certifiably correct computation. Below, max_size allows the user to pick the sample size. In this implementation we capped the number of tries to $10$ to avoid too many loops, which can be changed at will.

Notice an input parameter is the $L_r$-norm of $f$. Since it does not change, we compute it in advance once and save ourselves from computing it everytime this function loops.

In addition to the above, we introduced an "inner" input. This determines how far the newly found vector is from the previous ones. For example, an inner of $0.8$ forces the inner product between the new vector and the previous ones to be at most $0.8$. We introduced this controlled since vectors close to the previous ones (that is, inner product close to one) did not provide an appreciable improvement on the error. Therefore, if the new vector is not far enough from the previous ones, the function finds a new one.

In [15]:
def vec_finder(n, r, poly, lrnorm ,accu, max_size, inner, W, tries = 1):
    
    """
    Parameters:
    
    n: int. Number of variables.
    r: int. We need to specify here which L_r-norm we are using.
    poly: custom_polynomial. Polynomial in the class custom_polynomial. 
    lrnorm: float. L_r-norm of the given polynomial.
    accu: float. Number between 0 and 1 determining the level of accuracy.
    max_size: int. Number of points to sample.
    inner: float. Number between 0 and 1. 
    W: list. List of vectors corresponding to the orthogonal polynomials.
    tries: int. Auxiliary counter. Initial number of tries, set by default to start counting at 1.
    
    Outputs:
    
    v: list. Vector for which the inequality holds.
    fv: list. Value of the polynomial evaluated at the vector for which the inequality holds.
    q: custom_polynomial. Polynomial defined by the vector v, that is, <v,x>^d.
    """
    
    deg = poly.degree
    V = sample_spherical(n, max_size)
    
    if W != []:
        inner_product = np.absolute(np.matmul(V,np.array(W).T))
        V = V[np.max(inner_product,axis = 1) < inner,:]
        
    
    fV = parallelized_evaluation(poly,V)
    
    abs_fV = np.absolute(fV)
    
    max_position = np.argmax(abs_fV)
    v = V[max_position]
    fv = fV[max_position]
    abs_fv = np.max(abs_fV)
    
    max_tries = 10
    
    if 0.5*lrnorm <= abs_fv:
        q = custom_polynomial(1,v,deg)
        return list(v), fv, q
    else:
        tries += 1
        print("The vectors do not meet the requirement")
        if tries <= max_tries:
            print("Starting the next try, tries remaining: {}".format(max_tries - tries + 1))
            vec_finder(n, r, poly, lrnorm ,accu, max_size, inner, W,tries)
        else:
            raise Exception("Maximum number of tries exceeded")

## Orthogonal basis and orthogonal projection

We define an $L_r$-norm for a general symmetric tensor. This is due to the fact that we will need to compute the $L_r$-norm of the difference between $f$ and its orthogonal projection onto a subspace.

In [16]:
def Lr_norm(X, WX, n, r, poly):

    """
    Parameters:
    
    X: list. List of elements in X corresponding to the quadrature formula.
    WX: list. List of elements in WX corresponding to the quadrature formula.
    r: int. Point at which the polynomial is evaluated.
    
    Output:
    
    return: float. L_r-norm of the polynomial.
    """
    
    c = sphere_surface_area(n-1)
    deg = poly.degree
        
    evaluations = parallelized_evaluation(poly,X)
        
    evaluations = np.array(evaluations)**r
    
    return ((1/c) * np.sum(WX*(evaluations)))**(1/r)

Since we need the difference between two polynomials, we define such function.

In [17]:
def polynomial_difference(polynomial, new_expr):

    """
    Parameters:
    
    polynomial: custom_polynomial. First polynomial.
    new_expr: custom_polynomial. Second polynomial to be subtracted from the first one.
    
    Output:
    
    new_poly: custom_polynomial. Polynomial difference of the two polynomial above.
    """

    new_poly = copy.copy(polynomial)
    new_poly.expr -= new_expr
    new_poly.f = lambdify(list(new_poly.variables),new_poly.expr)
    
    return new_poly

The add_vector function does the following:
* Determines if the polynomial given by the found vector is orthogonal with respect to the previous ones.
* If orthogonal, then it adds it to the set of orthogonal plynomials in $P_{n,d}$ and projects $f$ onto the subspace given by the set.
* Computes the error. That is, the $L_r$-norm of the difference between $f$ and its orthogonal projection.

In [18]:
def add_vector(W,fW,Q,X,WX,n,r,poly,lrnorm,max_size,inner,accu):

    """
    Parameters:
    
    W: list. List of vectors for which their related polynomials are orthogonal.
    fW: list. List of evaluation of f at the vectors from W.
    Q: list. List of orthogonal polynomials given by the vectors in W.
    X: list. List of elements in X corresponding to the cubature formula.
    WX: list. List of elements in WX corresponding to the cubature formula.
    n: int. Number of variables.
    r: int. We need to specify here which L_r-norm we are using.
    poly: custom_polynomial. Polynomial in the class custom_polynomial. 
    lrnorm: float. L_r-norm of the given polynomial.
    max_size: int. Number of points to sample.
    inner: float. Number between 0 and 1.  
    accu: float. Number between 0 and 1 determining the level of accuracy.
    
    Outputs:
    
    W: list. List of vectors for which their related polynomials are orthogonal, including the last found.
    fW: list. List of evaluation of f at the vectors from W, including the last found.
    Q: list. List of orthogonal polynomials given by the vectors in W, including the last found.
    error: float. L_r-norm of the difference between f and its projection.
    projection_poly: custom_polynomial. Polynomial resulting from the projection of f onto the subspace.
    """

    v, fv, q = vec_finder(n, r, poly, lrnorm ,accu, max_size, inner, W)
    deg = poly.degree
    new_W = W + [v]
    #W.append(v)
    matrix = np.power((np.matrix(new_W)@np.matrix(new_W).T),deg)
    rank = np.linalg.matrix_rank(matrix)
    if rank != len(matrix):
        print("The matrix is singular")
        print("Computing a new vector")
        add_vector(W,fW,Q,X,WX,n,r,poly,lrnorm,max_size,inner,accu)
    else:
        W.append(v)
        fW.append(fv)
        Q.append(q)
        inv_matrix = np.linalg.inv(matrix)
        B = np.array((inv_matrix*(np.matrix(fW).reshape(len(matrix),1)))).ravel()
        projection_poly = 0
        for i in range(len(Q)):
            projection_poly += B[i]* Q[i].expr 
            
        poly_diff = polynomial_difference(poly, projection_poly)
            
        error = Lr_norm(X, WX, n, r, poly_diff)
        
        return W, fW, Q, error, projection_poly      

## Greedy Aproximation via Energy Increment

In [19]:
def greedy_approximation(n,r,poly,accu,inner,eps,tol=1e-03, max_iter = 1000):

    """
    Parameters:
    
    n: int. Number of variables.
    r: int. We need to specify here which L_r-norm we are using.
    poly: custom_polynomial. Polynomial in the class custom_polynomial. 
    accu: float. Number between 0 and 1 determining the level of accuracy.
    inner: float. Number between 0 and 1.
    eps: float. The improvement of the error we want at each iteration to keep the found vector.
    tol: float. Tolerance level to stop the process. Set by default to 1e-06.
    max_iter: int. Maximum number of iterations we want to compute. Set by default to 1,000.
    
    Outputs:
    
    projection_poly: custom_polynomial. Polynomial resulting from the projection of f onto the subspace.
    """
    
    deg = poly.degree
    max_vec = comb(n+deg-1 , deg)
    W = []
    fW = []
    Q = []
    
    print("Computing the Spherical Quadrature")
    init = time.time()
    X,WX = spherical_quadrature(n,deg)
    end = time.time()
    print("Elapsed time for the Spherical Quadrature: {} seconds".format(round(end - init)))

    lrnorm = poly.Lr_norm(X,WX,r)
    
    size = sample_size(n, deg, r, accu)
    # We cap here the sample size to 100,000 to be able to run the code in our computer.
    max_size = min(size,100000)
    error = np.inf
    print("The global threshold is: {}".format(eps))
    print("The size of a basis for this space is: {}".format(max_vec))
    iteration = 1
    while error >= eps:
        init = time.time()
        old_error = error
        W, fW, Q, error, projection_poly = add_vector(W,fW,Q,X,WX,n,r,poly,lrnorm,max_size,inner,accu)
        difference = old_error - error
        #print("The error is now: {}".format(error))
        #print("The difference between this iteration and the last one is: {}".format(difference))
        end = time.time()
        print("Elapsed time of the loop: {} seconds".format(end-init))
        
        if difference <= tol:
            error = old_error
            W = W[:-1]
            fW = fW[:-1]
            Q = Q[:-1]
            iteration +=1
            if iteration >= max_iter:
                print("Maximum number of iteration reached")
                break
        else:
            print("Vector found, {}".format(W[-1]))
            print("The error is now: {}".format(error))
            print("The difference between this iteration and the last one is: {}".format(difference))
            print("The number of vectors is {}".format(len(W)))
        
        if len(W) >= max_vec:
            print("Number of iteration exceeded")
            break
        

    return projection_poly    

##### Example

In [20]:
N = 10
n = 4
deg = 4
r = 4
accu = 0.95
inner = 0.8
eps = 0.3

vectors = sample_spherical(n,N)
consts = np.random.randn(N)
polynomial_list = [custom_polynomial(consts[i],vectors[i],deg) for i in range(N)]
poly = polynomial_sum(polynomial_list, eps)

In [21]:
%%time
projection_poly = greedy_approximation(n,r,poly,accu,inner,eps)

Computing the Spherical Quadrature

Elapsed time for the Spherical Quadrature: 0 seconds

The global threshold is: 0.3

The size of a basis for this space is: 35

Elapsed time of the loop: 0.12776541709899902 seconds

Vector found, [0.5028181796192638, 0.6413789631864076, 0.45430651788724785, -0.3597394774436946]

The error is now: 0.7317681134900962

The difference between this iteration and the last one is: inf

The number of vectors is 1

Elapsed time of the loop: 0.12867999076843262 seconds

Vector found, [-0.4690399929139189, -0.13294094019107944, 0.45611823031735255, -0.7445027544882487]

The error is now: 0.41628790535750354

The difference between this iteration and the last one is: 0.31548020813259264

The number of vectors is 2

Elapsed time of the loop: 0.11982250213623047 seconds

Vector found, [-0.13442939687709798, -0.6787059281543715, -0.06535452256887671, 0.7190380982417878]

The error is now: 0.3683100830400754

The difference between this iteration and the last one is

In [22]:
projection_poly

-0.512030217734277*(-0.630004375519503*x1 - 0.178563396024585*x2 + 0.612648143432157*x3 - x4)**4 - 0.237857292657558*(-0.422105263091009*x1 + 0.322065255378532*x2 - 0.0549959677422835*x3 - x4)**4 - 0.155749622382464*(-0.186957265833074*x1 - 0.943908159823467*x2 - 0.0908915991081466*x3 + x4)**4 + 0.0256338068322886*(-0.0512080298652965*x1 - 0.335991149224829*x2 - 0.842068441500781*x3 + x4)**4 - 0.32729938408849*(0.783964252773796*x1 + x2 + 0.708327749993899*x3 - 0.560884435087313*x4)**4 - 0.11187055373594*(x1 + 0.531976530221288*x2 + 0.284853168360958*x3 + 0.110407820870872*x4)**4